# Project 3 Shakespearean Sonnets RNN Part 6


In [64]:
### Preprocessing Choices

# Tokenizing your words
# Split up sequencies
# hypenated words
# capitalization?
# punctuation

# What changed?

# What didn't work/what changed 

# Analysis on the dataset?

In [65]:
import random
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random

# Initial Processing
- I'm not actually changing anything about the given text I'm literally just tokenizing the input and then passing the training data into the LSTM model.
- I am making 5 char jumps between samples of 40
- I tried to use the Keras tokenizer but it wasn't recognizing all of the unique character so I wrote my own
- 

In [70]:
# Turn text file into training set

# training data should be 40 character sequences from the sonnets
# take all possible subsequences of 40 consectives from the dataset
# use semi-redundant requences-> pick sequences starting every n-th character


def get_char_repr(char_to_index, char):
    unique_words = char_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[char_to_index[char]] = 1
    return feature_representation 

def preprocess_init(text):
    # Convert text to dataset using semi-redundant sequences
    unique_chars = sorted(list(set(text)))
    skip = 5 
    char_len = 40
    vocab_dict = {}
    for i in range(len(unique_chars)):
        vocab_dict[unique_chars[i]] = i
    print("Dict")
    print(len(vocab_dict))
    
    #list of sets fo 40 characters
    sequences = [] 
    # individual final characters 
    characters = []
    
    # generate seqs of 40 chars by looping through whole thing
    for i in range(0, len(text)-40, skip):
        sequences.append(text[i:i+40]) # sequence
        characters.append(text[i+41]) # char 
    
    # need to reshape because LSTM is being moody and wants a 3D thing for x
    trainX = np.zeros((len(sequences), 40, len(unique_chars)))
    trainY = np.zeros((len(sequences), len(unique_chars)))
    # put 1s into the places where things fit the correct char
    for index in range(len(sequences)):
        for seq in range(len(sequences[0])):
            trainX[index, seq] = get_char_repr(vocab_dict, sequences[index][seq])
        trainY[index] = get_char_repr(vocab_dict, characters[index])
        
    return trainX,trainY

In [71]:
text = open(os.path.join('data/shakespeare.txt')).read()
print(type(text))
seqX, chars = preprocess_init(text)
unique_chars = sorted(list(set(text)))


<class 'str'>
Dict
71


In [72]:
print(chars.shape)
print(seqX.shape)

(19598, 71)
(19598, 40, 71)


In [73]:
# Train a character-based LSTM model

def train_rnn(trainX, characters):
    
    model = Sequential()
    # single layers of 100-200 LSTM units
    model.add(LSTM(150, input_shape=(40,71)))
    # fully connected dense output layer with a softmax nonlinearity
    model.add(Dense(71, activation='softmax'))

    # Train model to minimize categorical cross-entropy
    model.compile(optimizer = 'rmsprop', # unclear if we need this
                  loss="categorical_crossentropy",
                  metrics = ["accuracy"]    
                  # we want accuracy over 0.6 on training data
                 )

    # train for many epochs so loss converges
    print("Train model...")
    model.fit(trainX, characters, epochs = 20, batch_size = 32)
    

In [ ]:
train_rnn(seqX, chars)

Train model...
Epoch 1/20
  352/19598 [..............................] - ETA: 34s - loss: 3.6491 - accuracy: 0.1278

In [149]:

# implement temp parameter myself as part of the poem generation algo
# add a Lambda layer to LSTM during prediction or write a function


In [ ]:
### Poetry Generation 

# to generate poems, draw softmax samples from the trained model
# Mess with temperature parameter? to control variance of your sampled text
